<h2>STAT 331 Final Project</h2><br>
<h3>For the final project, I have taken 2 datasets which contain the IPO data for various Indian companies ranging from 2006 to 2021. The 1st dataset contains really detailed information about each IPO, mostly concerning the listing prices. The 2nd dataset is almost identical to 1st dataset but it contains the percentage of listing & current gains, which is relevant information regarding an IPO. Since both datasets have unique values which are relevant to the dataset, the objective of this project would be link these datasets together.</h3>
<h3>While doing the EDA on these datasets, I found that the names of the companies did not exactly match between them. The 1st data had the more formal name, which can be found on official documentation. However, the 2nd dataset had just the name of the company and sometimes only its abbreviation. <br>This led me to implement some string-matching, allowing me to link the data easily.</h3>


In [1]:
# Install commands for modules used in the project
!pip install recordlinkage
!pip install fuzzymatcher

In [2]:
# Import Statements
import pandas as pd
import fuzzymatcher
import recordlinkage

In [3]:
# Reading datasets
df_a = pd.read_csv('Indian_IPO_Data.csv')
df_b = pd.read_csv('IPO.csv')

In [4]:
# Checking datasets
print(df_a.head())
print(df_b.head())

   Srno                        Company_Name Bid_Opening_Date Bid_Closing_Date  \
0     1       Voltamp Transformers Limited      Aug 24, 2006     Aug 29, 2006   
1     2                Deep Industries Ltd      Aug 29, 2006      Sep 4, 2006   
2     3              KEW Industries Limted      Aug 28, 2006      Sep 1, 2006   
3     4  Action Construction Equipment Ltd       Sep 1, 2006      Sep 7, 2006   
4     5                    Atlanta Limited       Sep 1, 2006      Sep 7, 2006   

   Face_Value Bid_Price_From Bid_Price_To  Market_Lot Listing_at  \
0        10.0            295          345         NaN   BSE, NSE   
1        10.0             36           36       175.0        BSE   
2        10.0             30           30       170.0        NSE   
3        10.0            110          130        20.0   BSE, NSE   
4        10.0            130          150        45.0   BSE, NSE   

                                          Issue_size  ...  \
0  4,883,840 Eq Shares of ₹10 (aggregating 

<h3>Dataset 1 contained many columns which were mostly empty, had duplicated data from the other columns or contained NaN values. Therefore, I started the data pre-processing by dropping these columns, as they had no lasting importance on the IPO records.</h3>

In [5]:
# Dropping columns with Maximum 0, duplicated, and NaN values in dataset df_a
df_a = df_a.drop(['Pre_Issue_Share_Holding', 'BSE_SME_Listing_day_open', 'BSE_SME_Listing_day_low', 'BSE_SME_Listing_day_high', 'BSE_SME_Listing_day_last_trade', 'BSE_SME_Listing_day_volume', 'NSE_SME_Listing_day_open', 'NSE_SME_Listing_day_low', 'NSE_SME_Listing_day_high', 'NSE_SME_Listing_day_last_trade', 'NSE_SME_Listing_day_volume'], axis = 1)
df_a.head()

,Srno,Company_Name,Bid_Opening_Date,Bid_Closing_Date,Face_Value,Bid_Price_From,Bid_Price_To,Market_Lot,Listing_at,Issue_size,...,NSE_Listing_day_open,NSE_Listing_day_low,NSE_Listing_day_high,NSE_Listing_day_last_trade,NSE_Listing_day_volume,BSE_Listing_day_open,BSE_Listing_day_low,BSE_Listing_day_high,BSE_Listing_day_last_trade,BSE_Listing_day_volume
0,1,Voltamp Transformers Limited,"Aug 24, 2006","Aug 29, 2006",10.0,295,345,NaN,"BSE, NSE","4,883,840 Eq Shares of ₹10 (aggregating up to ...",...,370,370,430.7,416.45,9531676.0,399,375,430.9,417.05,8445000.0
1,2,Deep Industries Ltd,"Aug 29, 2006","Sep 4, 2006",10.0,36,36,175.0,BSE,"11,300,000 Eq Shares of ₹10 (aggregating up to...",...,NaN,NaN,NaN,NaN,NaN,49,27,49,27.9,9704917.0
2,3,KEW Industries Limted,"Aug 28, 2006","Sep 1, 2006",10.0,30,30,170.0,NSE,"7,000,000 Eq Shares of ₹10 (aggregating up to ...",...,NaN,NaN,NaN,NaN,NaN,35.5,23.1,36,23.6,6977994.0
3,4,Action Construction Equipment Ltd,"Sep 1, 2006","Sep 7, 2006",10.0,110,130,20.0,"BSE, NSE","4,600,000 Eq Shares of ₹10 (aggregating up to ...",...,180,180,210.9,193.65,7582512.0,200,185.45,210.85,193.45,5792222.0
4,5,Atlanta Limited,"Sep 1, 2006","Sep 7, 2006",10.0,130,150,45.0,"BSE, NSE","4,300,000 Eq Shares of ₹10 (aggregating up to ...",...,175.55,175.55,208.65,192.3,12867315.0,170,170,209.5,192.3,11203665.0


<h3>Dataset 2 (df_b) did not require any pre-processing, apart from renaming some columns for better readability & understanding.</h3>

In [6]:
# Renaming improper columns
df_b = df_b.rename(columns={'Listing  \t\t\t\t\t\t  Gains(%)': 'Listing Gains(%)', 'Current \t\t\t\t\t\t  Gains (%)': 'Current Gains(%)', 'Issue Size \t\t\t\t\t\t  (in crores)': 'Issue Size(Crores)'})
df_b.head()


,Date,IPO Name,Issue Size(Crores),QIB,HNI,RII,Total,Issue,Listing Open,Listing Close,Listing Gains(%),CMP,Current Gains(%)
0,29-07-2021,Tatva Chintan,500.00,2.55,9.78,13.36,9.50,1083,2111.8,2310.25,113.32,"2,268.50",109.46
1,23-07-2021,Zomato,9375.00,51.79,32.96,7.45,38.25,76,115.0,125.85,65.59,133.35,75.46
2,19-07-2021,Clean Science,1546.00,156.37,206.43,9.00,93.41,900,1784.4,1585.20,76.13,"1,682.80",86.98
3,19-07-2021,G R Infra,963.28,93.18,73.01,4.89,43.48,837,1700.0,1746.80,108.70,"1,730.85",106.79
4,07-07-2021,India Pesticide,800.00,2.64,1.36,4.40,3.25,296,360.0,335.45,13.33,337.4,13.99


<h3>In order to understand the difference in company names, I selected 2 records to see what can be determined from a glance</h3>

In [7]:
# Selecting the same company in both datasets for a comparision
X_a = df_a.iloc[1024]
Y_b = df_b.iloc[29]
print('Dataset 1 Example: \n')
print(X_a)
print('\n')
print('Dataset 2 Example: \n')
print(Y_b)

Dataset 1 Example: 

Srno                                                                              1025
Company_Name                                                Burger King India Limited 
Bid_Opening_Date                                                           Dec 2, 2020
Bid_Closing_Date                                                           Dec 4, 2020
Face_Value                                                                          10
Bid_Price_From                                                                      59
Bid_Price_To                                                                        60
Market_Lot                                                                         250
Listing_at                                                                    BSE, NSE
Issue_size                           135,000,000 Eq Shares of ₹10 (aggregating up t...
Fresh_issue                          75,000,000 Eq Shares of ₹10 (aggregating up to...
Offer_for_sale        

<h3>As it can be seen, the major difference is that one name is an official name, while the other is the commonly used name. This helps to shortlist which methods should be used for matching them.</h3>

<h3> For these datasets, I followed 2 approaches to see which one fits the best.</h3>
<h2> Approach 1: Fuzzymatcher </h2>
<h3> Fuzzymatcher is a very accurate approach for matching strings in mid-sized datasets as it leverages sqlite’s full text search capability to try to match records in two different DataFrames.</h3>

In [8]:
# Since the columns have different names, we need to define which columns to match for the left and right DataFrames.
left_on = ["Company_Name", "Listing_date", "Total_Subscription"]

right_on = ["IPO Name", "Date", "Total"]

# It takes several minutes to run on this data set

matched_results = fuzzymatcher.fuzzy_left_join(df_a,
                                            df_b,
                                            left_on,
                                            right_on,
                                            left_id_col='Qualified_Institutional_Buyers',
                                            right_id_col='QIB')

print(matched_results)

          best_match_score  __id_left  __id_right  Srno  \
0                -0.418904        NaN       48.64     1   
8                -0.428371        NaN        0.56     1   
17               -0.529803        NaN        0.56     1   
26               -0.546408        NaN       22.03     1   
39               -0.308435        NaN       48.64     1   
...                    ...        ...         ...   ...   
24870480         -0.573102        NaN        0.88  1024   
24870568         -0.634225        NaN       89.76  1024   
24870643         -0.824186        NaN       48.64  1024   
24870716          0.440586      86.64       86.64  1025   
24870728          0.166943     176.85      176.85  1026   

                                     Company_Name Bid_Opening_Date  \
0                   Voltamp Transformers Limited      Aug 24, 2006   
8                   Voltamp Transformers Limited      Aug 24, 2006   
17                  Voltamp Transformers Limited      Aug 24, 2006   
26         

<h3>The matched_results DataFrame contains all the data linked together as well as as best_match_score which shows the quality of the link. To understand what has been, I defined a subset of the columns rearranged in a more readable format.</h3>

In [9]:
# Reorder the columns to make viewing easier
cols = [
    "best_match_score", "Company_Name", "IPO Name", "Listing_date", "Date",
    "Total_Subscription", "Total"
]

# Best matches
matched_results[cols].sort_values(by=['best_match_score'], ascending=False).head(5)

,best_match_score,Company_Name,IPO Name,Listing_date,Date,Total_Subscription,Total
6098500,0.739374,VA Tech Wabag Ltd,Va Tech Wabag,"Wednesday, October 13, 2010",13-10-2010,36.22,36.22
11547718,0.629784,L&T Infotech Ltd,L&T Infotech,"Thursday, July 21, 2016",21-07-2016,11.69,11.69
6133022,0.590436,Sea TV Network Ltd,Sea TV Network,"Thursday, October 14, 2010",14-10-2010,9.58,9.58
11958772,0.551478,L&T Technology Services Ltd,L&T Technology,"Friday, September 23, 2016",23-09-2016,2.52,2.52
9526494,0.508927,VRL Logistics Ltd,VRL Logistics,"Thursday, April 30, 2015",30-04-2015,74.26,74.26


In [10]:
# Worst matches
matched_results[cols].sort_values(by=['best_match_score'], ascending=False).tail(5)

,best_match_score,Company_Name,IPO Name,Listing_date,Date,Total_Subscription,Total
24768056,NaN,Bodhi Tree Multimedia Ltd,NaN,"Wednesday, October 21, 2020",NaN,1.60,NaN
24802295,NaN,Shine Fashions (India) Ltd,NaN,"Monday, November 2, 2020",NaN,1.73,NaN
24802296,NaN,Shine Fashions (India) Ltd,NaN,"Monday, November 2, 2020",NaN,1.73,NaN
24836536,NaN,Net Pix Shorts Digital Media Limited,NaN,"Wednesday, December 2, 2020",NaN,1.77,NaN
24836537,NaN,Net Pix Shorts Digital Media Limited,NaN,"Wednesday, December 2, 2020",NaN,1.77,NaN


<h3> By following the best match score, we find that the threshold for mismatches is -0.134. This allows us to filter out incorrect & inaccurate data from the dataframe.</h3>

In [11]:
# Correct match threshold
matched_results[cols].query("best_match_score >= -0.134").sort_values(
    by=['best_match_score'], ascending=False)

,best_match_score,Company_Name,IPO Name,Listing_date,Date,Total_Subscription,Total
6098500,0.739374,VA Tech Wabag Ltd,Va Tech Wabag,"Wednesday, October 13, 2010",13-10-2010,36.22,36.22
11547718,0.629784,L&T Infotech Ltd,L&T Infotech,"Thursday, July 21, 2016",21-07-2016,11.69,11.69
6133022,0.590436,Sea TV Network Ltd,Sea TV Network,"Thursday, October 14, 2010",14-10-2010,9.58,9.58
11958772,0.551478,L&T Technology Services Ltd,L&T Technology,"Friday, September 23, 2016",23-09-2016,2.52,2.52
9526494,0.508927,VRL Logistics Ltd,VRL Logistics,"Thursday, April 30, 2015",30-04-2015,74.26,74.26
...,...,...,...,...,...,...,...
15417672,-0.107164,Bharat Road Network Limited,Bharat Road Net,"Monday, September 18, 2017",18-09-2017,1.81,1.75
24425416,-0.113822,UTI Asset Management Company Ltd,UTI AMC,"Monday, October 12, 2020",12-10-2020,2.31,2.31
6169322,-0.113868,Inventure Growth & Securities Ltd,Inventure Grow,"Thursday, August 4, 2011",04-08-2011,4.58,4.58
22712733,-0.118837,Metropolis Healthcare Limited,Metropolis,"Monday, April 15, 2019",15-04-2019,5.83,5.84


<h3> As we can see, there isn't much difference between the names so it should not be very difficult to link records. However, we can also see that some records do not exist in the 2nd dataset(df_b), which should be a problem for good matches.</h3>

<h2> Approach 2: Python Record Linkage Toolkit </h2>
<h3>The Python Record Linkage Toolkit provides another robust set of tools for linking data records and identifying duplicate records in your data.</h3>
<h3>Because the Record Linkage Toolkit has more configuration options, we need to perform a couple of steps to define the linkage rules. The first step is to create a indexer object:</h3>

In [12]:
# Build the indexer
indexer = recordlinkage.Index()
indexer.full()

<Index>

<h3>This WARNING points us to a difference between the record linkage library and fuzzymatcher. With record linkage, we have some flexibility to influence how many pairs are evaluated. By using full indexer all potential pairs are evaluated</h3>
<h3>The next step is to build up all the potential candidates to check:</h3>

In [13]:
#Number of total comparisions that will be made
candidates = indexer.index(df_a, df_b)
print(len(candidates))

270864


<h3> Next, we perform the comparison logic using Compare(). We can define several options for how we want to compare the columns of data. For this example, we look for an exact match of Qualified Institutional Buyers, which is a unique value for each IPO.

In [14]:
# Finding the comparisions between the values by providing certain specifications
compare = recordlinkage.Compare()
compare.exact('Qualified_Institutional_Buyers', 'QIB', label='Qualified_Institutional_Buyers')
compare.string('Company_Name',
            'IPO Name',
            threshold=0,
            label='Name of Company IPO')

# Compared dataframe

features = compare.compute(candidates, df_a, df_b)
features

Qualified_Institutional_Buyers  Name of Company IPO
0    0                                 0                  1.0
     1                                 0                  1.0
     2                                 0                  1.0
     3                                 0                  1.0
     4                                 0                  1.0
...                                  ...                  ...
1025 259                               0                  1.0
     260                               0                  1.0
     261                               0                  1.0
     262                               0                  1.0
     263                               0                  1.0

[270864 rows x 2 columns]

<h3> This dataFrame shows the results of all of the comparisons. Given the large number of records with no matches, it is a little hard to see how many matches we might have. We can sum up the individual scores to see about the quality of the matches.<h3>

In [15]:
# Score totals
features.sum(axis=1).value_counts().sort_index(ascending=False)

2.0       409
1.0    270455
dtype: int64

<h3> Thus, we can see that 270455 values have a match, while 409 values do not. To find the exact values that do not match, we need to create a lookup. However,it may not work as effectively due to the lack of features, which can be attributed to the medium sized dataset.</h3>
<h2>Therefore, we can see that while the Python Record Linkage Tool is better in terms of computational performance & flexibility, Fuzzymatcher is a better fit for our datasets as the tool is very suitable for mid-size datasets.</h2>

<h3> Moving ahead, we now link the datasets by using the matched_result dataframe.</h3>

In [16]:
# Defining the columns required for the new dataset
imp_cols = ['best_match_score', "Company_Name", "IPO Name", "Listing_date", 'Total_Subscription', 
            'Bid_Price_From', 'Bid_Price_To', 'Listing_at', 'Issue Size(Crores)', 'Listing Gains(%)', 'Current Gains(%)', 
            'Qualified_Institutional_Buyers', 'Retail_Individual_Investors']

# Creating the new dataset by filtering the values and dropping the convinience columns

new_df = matched_results[imp_cols].query("best_match_score >= -0.134").sort_values(
    by=['best_match_score'], ascending=False).reset_index()
new_df = new_df.drop(['index', 'best_match_score', 'IPO Name'], axis=1)
new_df

,Company_Name,Listing_date,Total_Subscription,Bid_Price_From,Bid_Price_To,Listing_at,Issue Size(Crores),Listing Gains(%),Current Gains(%),Qualified_Institutional_Buyers,Retail_Individual_Investors
0,VA Tech Wabag Ltd,"Wednesday, October 13, 2010",36.22,1230,1310,"BSE, NSE",472.60,-73.90,-71.76,36.13,8.55
1,L&T Infotech Ltd,"Thursday, July 21, 2016",11.69,705,710,"BSE, NSE",1242.00,-1.74,559.83,19.91,7.39
2,Sea TV Network Ltd,"Thursday, October 14, 2010",9.58,90,100,BSE,50.20,6.00,-98.02,1.66,7.45
3,L&T Technology Services Ltd,"Friday, September 23, 2016",2.52,850,860,"BSE, NSE",900.00,0.59,332.56,5.01,1.74
4,VRL Logistics Ltd,"Thursday, April 30, 2015",74.26,195,205,"BSE, NSE",473.88,43.07,48.12,58.22,7.92
...,...,...,...,...,...,...,...,...,...,...,...
1614,Bharat Road Network Limited,"Monday, September 18, 2017",1.81,195,205,"BSE, NSE",600.65,1.54,-83.02,1.33,5.69
1615,UTI Asset Management Company Ltd,"Monday, October 12, 2020",2.31,552,554,"BSE, NSE",2159.88,-13.97,88.10,3.34,2.32
1616,Inventure Growth & Securities Ltd,"Thursday, August 4, 2011",4.58,100,117,"BSE, NSE",81.90,-95.56,-96.98,0.25,8.66
1617,Metropolis Healthcare Limited,"Monday, April 15, 2019",5.83,877,880,"BSE, NSE",1200.18,9.04,224.43,8.88,2.15


<h3> In conclusion, by using fuzzymatcher & record linking, a complete dataset with accurate data & labels has been achieved.</h3> 
<h3>For the project's future scope, improvements in the dataset's quality can be made using merging techniques which may help enhance the dataset.</h3>